# Process `prachathai-67k` for [HuggingFace Datasets](https://github.com/huggingface/datasets)

This notebook performs a transformation of the `prachathai-67k` dataset which was scraped from the news site [Prachathai](https://prachatai.com/) for `huggingface/datasets`. We filtered out those articles with less than 500 characters of body text, mostly images and cartoons. It contains 67,889 articles wtih 51,797 tags from August 24, 2004 to November 15, 2018. The dataset was originally scraped by [@lukkiddd](https://github.com/lukkiddd) and cleaned by [@cstorm125](https://github.com/cstorm125).

This dataset is a part of [pyThaiNLP](https://github.com/PyThaiNLP/) Thai text [classification-benchmarks](https://github.com/PyThaiNLP/classification-benchmarks). For the benchmark, we selected the following tags with substantial volume that resemble **classifying types of articles**$^+$:

* `การเมือง` - politics
* `สิทธิมนุษยชน` - human rights
* `คุณภาพชีวิต` - quality of life
* `ต่างประเทศ` - international
* `สังคม` - social
* `สิ่งแวดล้อม` - environment
* `เศรษฐกิจ` - economics
* `วัฒนธรรม` - culture
* `แรงงาน` - labor
* `ความมั่นคง` - national security
* `ไอซีที` - ICT
* `การศึกษา` - education

This is a **multi-label classification task** and we use **F1-score** for each label as the benchmark. The benchmark include both **title classification** and **body classification**.

$^+$Note that Prachathai.com is a left-leaning, human-right-focused news site, and thus unusual news labels such as human rights and quality of life.

In [1]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from ast import literal_eval
from tqdm.auto import tqdm
tqdm.pandas()
from collections import Counter
from sklearn.model_selection import train_test_split

/Users/admin/venv/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
prachathai = pd.read_csv('prachathai-67k.csv')
print(prachathai.shape)
prachathai.head()

(67889, 5)


,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


## Labels

### Tag Distribution

In [4]:
all_labels = []
for i in tqdm(range(prachathai.shape[0])):
    all_labels+=literal_eval(prachathai.labels[i])

In [5]:
label_df = pd.DataFrame.from_dict(dict(Counter(all_labels)),orient='index').reset_index()
label_df.columns = ['label','cnt']
label_df.sort_values('cnt',ascending=False,inplace=True)
print(label_df.shape)
label_df.head(15)

(51797, 2)


,label,cnt
0,ข่าว,53448
2,การเมือง,39124
62,สิทธิมนุษยชน,15067
42,คุณภาพชีวิต,11316
17,บทความ,9571
18,ต่างประเทศ,8114
43,สังคม,7893
1,สิ่งแวดล้อม,7703
19,เศรษฐกิจ,5008
135,วัฒนธรรม,4070


In [6]:
label_df.tail(10)

,label,cnt
9388,ซามีร์ อามิน,1
28298,ตาดูดาว เท้าติดดิน,1
28296,พันธมิตรเพื่อปกป้องแม่น้ำโขง,1
28295,คำแถลงนโยบาย,1
28294,ชายแดนใต้ กระบวนการสันติภาพ พูโล BRN พล.อ.อกนิ...,1
28293,เทวาลัย,1
28292,เครื่องฉีดน้ำแรงดันสูง,1
28291,อักษรศาสตร์,1
28290,รอยพญานาค,1
51796,วันประกาศเอกราช,1


### Benchmark Labels

Out of over 50,000 tags, we selected 12 tags as our **benchmark labels**.

In [7]:
benchmark_labels = ['การเมือง','สิทธิมนุษยชน','คุณภาพชีวิต','ต่างประเทศ','สังคม',
                  'สิ่งแวดล้อม','เศรษฐกิจ','วัฒนธรรม','แรงงาน','ความมั่นคง',
                  'ไอซีที','การศึกษา']

def in_benchmark(x,label):
    res = 1 if label in literal_eval(x) else 0
    return(res)

In [8]:
for label in tqdm(benchmark_labels):
    prachathai[label] = prachathai.labels.map(lambda x: in_benchmark(x,label))

In [9]:
tags_en = [
    'politics',
    'human_rights',
    'quality_of_life',
    'international',
    'social',
    'environment',
    'economics',
    'culture',
    'labor',
    'national_security',
    'ict',
    'education',
]
prachathai.columns = ['url','date','title','body_text','labels'] + tags_en
prachathai = prachathai.drop('labels',1)
prachathai

,url,date,title,body_text,politics,human_rights,quality_of_life,international,social,environment,economics,culture,labor,national_security,ict,education
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,0,0,0,0,0,1,0,0,0,0,0,0
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,0,0,0,0,0,1,0,0,0,0,0,0
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,0,0,0,0,0,1,0,0,0,0,0,0
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,1,0,0,0,0,0,0,0,0,0,0,0
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67884,https://prachatai.com/print/79624,2018-11-15 12:39,ชราธิปไตยในสังคมพุทธไทย,พุทธศาสนาเถรวาทมีขนบที่ต้องเคารพคนแก่ เพราะเชื...,1,1,0,0,0,0,0,1,0,0,0,0
67885,https://prachatai.com/print/79626,2018-11-15 18:51,เวทีอาเซียนสนใจสถานการณ์โรฮิงญา บังกลาเทศเลื่อ...,ประเด็นวิกฤตชาวโรฮิงญาเป็นที่ถกเถียงในเวทีประช...,1,1,0,1,0,0,0,0,0,0,0,0
67886,https://prachatai.com/print/79625,2018-11-15 12:47,ยุติวงจรผลัดกันเกาหลัง ด้วยการยื่นบัญชีทรัพย์สินฯ,จากกรณีที่ราชกิจจานุเบกษาได้เผยแพร่ประกาศของคณ...,1,0,0,0,0,0,1,0,0,0,0,0
67887,https://prachatai.com/print/79628,2018-11-15 20:45,รัฐสวัสดิการ 101 กับ ภาคภูมิ แสงกนกกุล : การจั...,ทำความเข้าใจรัฐสวัสดิการแบบ 101 ทั้งความเป็นมา...,1,0,1,0,0,0,1,0,1,0,0,0


In [11]:
#some processing
prachathai['title'] = prachathai['title'].map(lambda x: x.strip())
prachathai['body_text'] = prachathai['body_text'].map(lambda x: x.strip())
# prachathai['nb_title'] = prachathai['title'].map(lambda x: len(word_tokenize(x)))
# prachathai['nb_body_text'] = prachathai['body_text'].map(lambda x: len(word_tokenize(x)))

## Splits

In [12]:
train_valid, test = train_test_split(prachathai,test_size=0.1,random_state=1412)
train, valid = train_test_split(train_valid,test_size=0.11,random_state=1412)
train.shape, valid.shape, test.shape

((54379, 16), (6721, 16), (6789, 16))

In [13]:
train.iloc[:,4:].sum()

politics             31401
human_rights         12061
quality_of_life       9037
international         6432
social                6321
environment           6157
economics             3994
culture               3279
labor                 2905
national_security     2865
ict                   2326
education             2093
dtype: int64

In [14]:
valid.iloc[:,4:].sum()

politics             3852
human_rights         1458
quality_of_life      1144
international         828
social                782
environment           764
economics             487
culture               388
labor                 375
national_security     339
ict                   285
education             248
dtype: int64

In [15]:
test.iloc[:,4:].sum()

politics             3842
human_rights         1511
quality_of_life      1127
international         834
social                789
environment           772
economics             519
culture               398
labor                 350
national_security     338
ict                   292
education             255
dtype: int64

In [18]:
#save
train.to_json('huggingface/train.jsonl',orient='records',lines=True)
valid.to_json('huggingface/valid.jsonl',orient='records',lines=True)
test.to_json('huggingface/test.jsonl',orient='records',lines=True)

In [22]:
#view
import json
with open('data/test.jsonl', 'r') as f:
    x = f.readlines()
json.loads(x[0])

{'url': 'https://prachatai.com/print/62490',
 'date': '2015-11-17 18:14',
 'title': 'แฮคเกอร์ Anonymous ลั่นทำสงครามไซเบอร์ครั้งใหญ่สุดกับกลุ่ม IS',
 'body_text': '17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์กับกลุ่มหัวรุนแรงหลังจากกลุ่ม IS ออกมาประกาศว่าเป็นผู้อยู่เบื้องหลังการโจมตีกรุงปารีสในคืนวันศุกร์ที่ผ่านมา\n\n\nภาพในคลิปใน YouTube โฆษกของกลุ่มแฮคเกอร์สวมหน้ากากที่เป็นสัญลักษณ์ของกลุ่มได้ออกมาอ่านแถลงเป็นภาษาฝรั่งเศส มีใจความว่า จากการโจมตีของกลุ่ม IS ในกรุงปารีส กลุ่ม Anonymous ทั่วโลกจะตามล่ากลุ่ม IS เหมือนที่เคยทำตอนที่มีการโจมตีสำนักพิมพ์ Charlie Hebdo และครั้งนี้จะเป็นปฏิบัติการโจมตีครั้งใหญ่ที่สุดของกลุ่ม Anonymous เลย นอกจากนี้กลุ่ม Anonymous ยังแสดงความเสียใจต่อครอบครัวผู้สูญเสียในเหตุการณ์ครั้งนี้\nกลุ่ม Anonymous เคยประกาศสงครามกับกลุ่ม IS หลังจากการโจมตีสำนักพิมพ์ Charlie Hebdo ที่ฝรั่งเศสเมื่อต้นปีที่ผ่านมา ซึ่งครั้งนั้นกลุ่ม Anonymous อ้างว่าได้ระงับบัญชีผู้ใช้งานที่เกี่ยวข้องกับ IS ไปหลายพันบัญชี (อ่านรายละเอียดเพิ่มเติม จากBlogno